In [1]:
import random
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.datasets.samples_generator import make_blobs
from fuzzy_cmeans import FCM 
import pandas as pd

In [2]:
clusters=4

In [3]:
features= pd.read_csv('dataset for 9.csv') 
features.label.unique()

array(['alternative', 'blues', 'electronic', 'folkcountry', 'funksoulrnb',
       'jazz', 'pop', 'raphiphop', 'rock'], dtype=object)

In [4]:
features=features.drop(features[features.label=='rock'].index)
features=features.drop(features[features.label=='raphiphop'].index)
features=features.drop(features[features.label=='pop'].index)
features=features.drop(features[features.label=='jazz'].index)
features=features.drop(features[features.label=='funksoulrnb'].index)
features.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,50_Minutes-Colours.wav,0.370957,0.191483,1554.272672,2272.273786,3586.444289,0.039205,-128.463409,117.550293,32.394691,...,4.688472,-7.322381,-0.384554,-0.297131,-0.951402,-5.560275,2.688139,-3.069870,2.455562,alternative
1,50_Minutes-Nightwalk.wav,0.301646,0.118957,892.314417,1477.003664,1649.473129,0.033195,-228.945496,162.367874,25.274275,...,-2.929274,-5.459329,-3.360488,-5.309353,-2.012183,-8.864375,-0.077231,-6.165314,0.011491,alternative
2,50_Minutes-The_Great_Wall.wav,0.375948,0.203939,1270.664430,1838.236128,2720.278635,0.043340,-114.051369,133.875641,-2.535841,...,1.340597,-1.123214,4.357597,-1.194922,-6.105379,-10.633196,-7.830831,-9.174461,1.239748,alternative
3,Abigail_Lapell-Dress_Rehearsal.wav,0.288743,0.124793,1946.785743,2323.686436,4007.160408,0.080200,-159.387787,104.808685,-9.930721,...,0.982948,-14.001296,2.717434,-12.282285,-2.516945,-7.880888,-3.486890,-5.966151,-2.050257,alternative
4,Albino_Catfish-Ice_Cream.wav,0.291857,0.154149,2477.062284,2280.878176,4829.267342,0.135053,-123.162804,69.661911,-16.024954,...,-8.382612,-10.428762,-10.952260,-3.459139,-4.728041,-5.685522,-10.800599,-2.911047,-0.371204,alternative


In [5]:
x= features.iloc[:,1:-1]
y=features.iloc[:,-1]
x=np.asarray(x)

In [6]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
y=encoder.fit_transform(y)

In [7]:
def _k_init(X, n_clusters, x_squared_norms, random_state, n_local_trials=None):
    """Init n_clusters seeds according to k-means++
    Parameters
    ----------
    X : array or sparse matrix, shape (n_samples, n_features)
        The data to pick seeds for. To avoid memory copy, the input data
        should be double precision (dtype=np.float64).
    n_clusters : integer
        The number of seeds to choose
    x_squared_norms : array, shape (n_samples,)
        Squared Euclidean norm of each data point.
    random_state : int, RandomState instance
        The generator used to initialize the centers. Use an int to make the
        randomness deterministic.
        See :term:`Glossary <random_state>`.
    n_local_trials : integer, optional
        The number of seeding trials for each center (except the first),
        of which the one reducing inertia the most is greedily chosen.
        Set to None to make the number of trials depend logarithmically
        on the number of seeds (2+log(k)); this is the default.
    Notes
    -----
    Selects initial cluster centers for k-mean clustering in a smart way
    to speed up convergence. see: Arthur, D. and Vassilvitskii, S.
    "k-means++: the advantages of careful seeding". ACM-SIAM symposium
    on Discrete algorithms. 2007
    Version ported from http://www.stanford.edu/~darthur/kMeansppTest.zip,
    which is the implementation used in the aforementioned paper.
    """
    n_samples, n_features = X.shape

    centers = np.empty((n_clusters, n_features), dtype=X.dtype)

    assert x_squared_norms is not None, 'x_squared_norms None in _k_init'

    # Set the number of local seeding trials if none is given
    if n_local_trials is None:
        # This is what Arthur/Vassilvitskii tried, but did not report
        # specific results for other than mentioning in the conclusion
        # that it helped.
        n_local_trials = 2 + int(np.log(n_clusters))

    # Pick first center randomly
    center_id = random_state.randint(n_samples)
    if sp.issparse(X):
        centers[0] = X[center_id].toarray()
    else:
        centers[0] = X[center_id]

    # Initialize list of closest distances and calculate current potential
    closest_dist_sq = euclidean_distances(
        centers[0, np.newaxis], X, Y_norm_squared=x_squared_norms,
        squared=True)
    current_pot = closest_dist_sq.sum()

    # Pick the remaining n_clusters-1 points
    for c in range(1, n_clusters):
        # Choose center candidates by sampling with probability proportional
        # to the squared distance to the closest existing center
        rand_vals = random_state.random_sample(n_local_trials) * current_pot
        candidate_ids = np.searchsorted(stable_cumsum(closest_dist_sq),
                                        rand_vals)

        # Compute distances to center candidates
        distance_to_candidates = euclidean_distances(
            X[candidate_ids], X, Y_norm_squared=x_squared_norms, squared=True)

        # Decide which candidate is the best
        best_candidate = None
        best_pot = None
        best_dist_sq = None
        for trial in range(n_local_trials):
            # Compute potential when including center candidate
            new_dist_sq = np.minimum(closest_dist_sq,
                                     distance_to_candidates[trial])
            new_pot = new_dist_sq.sum()

            # Store result if it is the best local trial so far
            if (best_candidate is None) or (new_pot < best_pot):
                best_candidate = candidate_ids[trial]
                best_pot = new_pot
                best_dist_sq = new_dist_sq

        # Permanently add best center candidate found in local tries
        if sp.issparse(X):
            centers[c] = X[best_candidate].toarray()
        else:
            centers[c] = X[best_candidate]
        current_pot = best_pot
        closest_dist_sq = best_dist_sq

    return centers

In [8]:
# x_squared_norms = np.row_norms(x, squared=True)

from sklearn.utils.extmath import row_norms
from sklearn.utils.extmath import row_norms, squared_norm, stable_cumsum
import scipy.sparse as sp
from sklearn.metrics.pairwise import euclidean_distances
 
c = _k_init(x,clusters, row_norms(x, squared=True), np.random.RandomState())
c

array([[ 4.21118766e-01,  1.87656492e-01,  2.71678272e+03,
         2.59706857e+03,  5.64378308e+03,  1.28595730e-01,
        -5.13037911e+01,  7.60298309e+01, -2.85786796e+00,
         3.08462448e+01,  7.74103212e+00,  1.79223404e+01,
         7.05444872e-01,  1.22092924e+01, -6.22890902e+00,
         3.92800617e+00, -2.95216948e-01,  5.67567587e-01,
        -6.42214155e+00, -3.75519061e+00, -4.57992029e+00,
        -1.55728325e-01, -3.68387675e+00,  6.48617804e-01,
        -6.72755384e+00, -2.33831763e+00],
       [ 4.22161996e-01,  2.62231857e-01,  1.58381222e+03,
         1.89711427e+03,  3.20569360e+03,  6.12320440e-02,
        -7.52895508e+01,  1.31155014e+02, -1.08565540e+01,
         4.04202652e+01,  1.93183346e+01,  2.85843563e+00,
         1.19162464e+01,  1.46439323e+01, -8.10121250e+00,
         1.13938761e+01, -2.92349458e+00,  1.84483612e+00,
        -1.64606154e+00,  4.53950500e+00, -4.66522884e+00,
         4.11143637e+00, -3.73627019e+00,  1.36517680e+00,
        -2.51

In [9]:
fuzzy_cmeans2 = FCM(n_clusters=clusters)
fuzzy_cmeans2.centers=c

fuzzy_cmeans2.fit(x)
centers = fuzzy_cmeans2.centers


label=fuzzy_cmeans2.predict(x)

label.shape

(600,)

In [10]:
np.unique(label)
ny=label

In [11]:

from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics.cluster import normalized_mutual_info_score

print(adjusted_rand_score(y,ny))
print(mean_squared_error(y,ny))
print(normalized_mutual_info_score(y,ny))


0.006963852944425127
2.275
0.02006874774880021


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)


In [12]:
test=pd.DataFrame()
test['y']=y
test['ny']=ny
test.head()

,y,ny
0,0,1
1,0,3
2,0,3
3,0,1
4,0,2


In [13]:
import itertools 
perm = itertools.permutations(np.unique(y)) 
_totalperm=[]
  
for i in list(perm): 
    _totalperm.append(list(i)) 

In [14]:
from sklearn import metrics as m
accuracy=np.zeros((len(_totalperm)))
values=[]

for i in range(len(_totalperm)): 
    print(i)
    newcolvals=np.zeros((test.shape[0]))
    for j in range(test.shape[0]):
        newcolvals[j]=(_totalperm[i])[test.ny[j]] 
        
    accval=m.accuracy_score(test.y,newcolvals)
    accuracy[i]=(accval)
    values.append(list(_totalperm[i]))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23


In [15]:
np.max(accuracy)

0.31